## LLM Fine Tuning

#### Install Dependancies

In [ ]:
# Utils
!pip install -q \
    numpy \
    pandas \
    scikit-learn \
    pyyaml \
    python-dotenv \
    tqdm


In [ ]:
# Create .env file with API key
import os
from google.colab import userdata
# Write .env file
# with open('.env', 'w') as f:
#     # Add the secrets if needed
#     f.write('GOOGLE_API_KEY=<api_key_here>\n')
#     f.write('HF_TOKEN=<api_key_here>\n')

# print(" .env file created")

with open('.env', 'w') as f:
    # Add the secrets if needed
    f.write(f'GOOGLE_API_KEY={userdata.get('GOOGLE_API_KEY')}\n')
    f.write(f'HF_TOKEN={userdata.get('HF_TOKEN')}\n')

print(" .env file created")

